In [1]:
from BlackBox.lmart.lMart_utils import LMARTGridsearch
import pandas as pd

# Grid-search

In [2]:
lMart_parameter = dict(
    boosting_type=["dart", "gbdt"],
    num_leaves=[2, 5, 10, 20, 30],
    max_depth=[-1],
    n_estimators=[75, 80, 100, 150, 200],
    learning_rate=[0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)

gs = LMARTGridsearch(path_dataset="../../outputs/scores.csv", random_state=841)
# best_ = gs.grid_search(lMart_parameter)

In [2]:
# best_model = best_[0]
# print(best_)

(LGBMRanker(boosting_type='dart', class_weight='balanced', force_row_wise=True,
            importance_type='gain', learning_rate=0.2, metric='ndcg',
            n_estimators=80, num_leaves=5, objective='lambdarank',
            reg_lambda=0.0003, verbose=-1),
 {'boosting_type': 'dart',
  'learning_rate': 0.2,
  'max_depth': -1,
  'n_estimators': 80,
  'num_leaves': 5,
  'reg_lambda': 0.0003},
 0.806521033135338)

# Metric evaluation

In [3]:
best_par = {'boosting_type': 'dart', 'learning_rate': 0.2, 'max_depth': -1,
            'n_estimators': 80, 'num_leaves': 5, 'reg_lambda': 0.0003}
best_model = gs.fit(**best_par)

In [4]:
# nDCG on test-set
nDCG_train = gs.eval_model(model=best_model, df=gs.train, qIds=gs.qIds_train)
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, qIds=gs.qIds_val)
nDCG_test = gs.eval_model(model=best_model, df=gs.test, qIds=gs.qIds_test)

print("nDCG@15")
print("Training set", round(nDCG_train,4))
print("Validation set", round(nDCG_valid,4))
print("Test set", round(nDCG_test,4))

nDCG@15
Training set 0.8
Validation set 0.8065
Test set 0.8008


# Example of Job-offer

In [5]:
qId= 1
job_curricula = gs.test[gs.test["qId"] == qId]

y_pred = best_model.predict(job_curricula.iloc[:,2:13])

y_pred = pd.DataFrame(y_pred, index=job_curricula.index, columns=["lambdas"])
dt_final = pd.merge(job_curricula, y_pred, left_index=True, right_index=True)

In [6]:
dt_final.sort_values("labels",ascending=False)["labels"].head(15)

1401    4
1274    4
990     4
915     4
1371    4
1373    4
1160    4
928     3
1086    3
1363    3
1088    3
1091    3
934     3
893     3
1085    3
Name: labels, dtype: int64

In [7]:
dt_final.sort_values("lambdas",ascending=False)["labels"].head(15)

1274    4
968     3
990     4
1271    3
920     3
1403    2
1165    2
946     2
876     3
997     3
1088    3
1121    2
1086    3
1442    2
962     2
Name: labels, dtype: int64